In [1]:
from typing import cast

import numpy as np
import pandas as pd
from scipy.io.arff import loadarff

%load_ext autoreload
%autoreload 2
import ml_project.helpers as h

pd.set_option("future.no_silent_downcasting", True)

Data files can be downloaded from the nextcloud. VPN2016/Scenario B-ARFF/TimeBasedFeatures-Dataset-15s.arff

In [2]:
raw_data = loadarff("../data/15s.arff")

In [3]:
orig = pd.DataFrame(raw_data[0])
orig = orig.rename(columns={"class1": "clas"})

classes = sorted(orig["clas"].unique())
class_map = {c: index + 1 for index, c in enumerate(classes)}
class_map_rev = {
    index + 1: f"{c.decode()} ({index + 1})" for index, c in enumerate(classes)
}

orig["clas"] = orig["clas"].replace(class_map).astype(int)
orig["clas"].unique(), class_map_rev

(array([ 2,  3,  4,  6,  7,  9, 12, 10, 11, 13, 14,  1,  8,  5]),
 {1: 'BROWSING (1)',
  2: 'CHAT (2)',
  3: 'FT (3)',
  4: 'MAIL (4)',
  5: 'P2P (5)',
  6: 'STREAMING (6)',
  7: 'VOIP (7)',
  8: 'VPN-BROWSING (8)',
  9: 'VPN-CHAT (9)',
  10: 'VPN-FT (10)',
  11: 'VPN-MAIL (11)',
  12: 'VPN-P2P (12)',
  13: 'VPN-STREAMING (13)',
  14: 'VPN-VOIP (14)'})

In [4]:
orig_features = cast(list[str], sorted(orig.drop(columns="clas").columns))
orig_features_std = list(filter(lambda x: x.startswith("std_"), orig_features))
orig_features_not_std = list(filter(lambda x: not x.startswith("std_"), orig_features))
orig_features_std, orig_features_not_std

(['std_active', 'std_flowiat', 'std_idle'],
 ['duration',
  'flowBytesPerSecond',
  'flowPktsPerSecond',
  'max_active',
  'max_biat',
  'max_fiat',
  'max_flowiat',
  'max_idle',
  'mean_active',
  'mean_biat',
  'mean_fiat',
  'mean_flowiat',
  'mean_idle',
  'min_active',
  'min_biat',
  'min_fiat',
  'min_flowiat',
  'min_idle',
  'total_biat',
  'total_fiat'])

In [5]:
def class_fraction(df_num: pd.DataFrame, df_den: pd.DataFrame) -> pd.DataFrame:
    population = df_num.groupby("clas").size()
    fraction = population * 1000 // df_den.groupby("clas").size() / 10
    return pd.DataFrame({"population": population, "fraction": fraction}).rename(
        class_map_rev
    )

In [6]:
class_fraction(orig, orig)

,population,fraction
clas,,
BROWSING (1),2500,100.0
CHAT (2),890,100.0
FT (3),1018,100.0
MAIL (4),249,100.0
P2P (5),1000,100.0
STREAMING (6),482,100.0
VOIP (7),2826,100.0
VPN-BROWSING (8),2500,100.0
VPN-CHAT (9),1196,100.0


First, completely missing values

In [7]:
with_na = orig.loc[orig.apply(lambda x: x.isna().any(), axis=1)]
class_fraction(with_na, orig)

,population,fraction
clas,,
BROWSING (1),NaN,NaN
CHAT (2),NaN,NaN
FT (3),NaN,NaN
MAIL (4),NaN,NaN
P2P (5),NaN,NaN
STREAMING (6),NaN,NaN
VOIP (7),NaN,NaN
VPN-BROWSING (8),NaN,NaN
VPN-CHAT (9),NaN,NaN


In [8]:
h.describe(orig)

,count,mean,std,min,25%,50%,75%,max
duration,18758.0,16.000000,16.400000,-inf,12.7,16.2,16.5,20.2
total_fiat,18758.0,13.300000,14.600000,NaN,1.9,4.1,7.4,17.4
total_biat,18758.0,13.300000,14.600000,NaN,0.6,3.0,7.4,17.5
min_fiat,18758.0,15.000000,16.100000,NaN,10.1,12.5,15.2,19.5
min_biat,18758.0,14.800000,16.000000,NaN,10.1,12.5,15.2,20.2
max_fiat,18758.0,13.800000,14.900000,-inf,9.3,10.8,13.1,18.8
max_biat,18758.0,13.700000,14.700000,-inf,8.6,10.4,13.0,17.5
mean_fiat,18758.0,13.600000,15.200000,-inf,4.7,9.3,13.3,19.1
mean_biat,18758.0,13.300000,14.700000,-inf,-inf,9.1,13.1,18.4
flowPktsPerSecond,18758.0,7.600000,9.800000,-inf,0.9,2.4,4.6,13.8


Does not look great.

The data has a lot of 0 and negative values. It does not fit in how the features are described in the paper.

And while `std_* == 0` could (no) be a valid value, everything else looks just bad.

In [9]:
some_missing_cond = (orig[orig_features_not_std] <= 0).apply("any", axis=1)
some_missing = orig.loc[some_missing_cond]
h.describe(some_missing)

,count,mean,std,min,25%,50%,75%,max
duration,13133.0,15.800000,15.900000,-inf,12.0,15.8,16.5,19.5
total_fiat,13133.0,13.600000,14.700000,NaN,1.3,4.2,8.6,17.4
total_biat,13133.0,13.600000,14.800000,NaN,NaN,2.8,8.5,17.5
min_fiat,13133.0,14.000000,15.400000,NaN,7.6,10.6,13.1,19.5
min_biat,13133.0,13.900000,14.800000,NaN,NaN,10.6,13.1,17.5
max_fiat,13133.0,13.700000,14.800000,-inf,6.3,9.8,11.9,18.4
max_biat,13133.0,13.600000,14.800000,-inf,-inf,9.8,11.6,17.5
mean_fiat,13133.0,11.800000,14.500000,-inf,-inf,7.9,9.6,18.9
mean_biat,13133.0,11.100000,12.700000,-inf,-inf,7.7,9.8,16.1
flowPktsPerSecond,13133.0,7.900000,10.000000,-inf,1.9,4.1,4.7,13.8


In [10]:
class_fraction(some_missing, orig)

,population,fraction
clas,,
BROWSING (1),822,32.8
CHAT (2),408,45.8
FT (3),825,81.0
MAIL (4),172,69.0
P2P (5),940,94.0
STREAMING (6),361,74.8
VOIP (7),2813,99.5
VPN-BROWSING (8),1179,47.1
VPN-CHAT (9),445,37.2


In [11]:
none_missing = orig.loc[~some_missing_cond]
none_missing

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,...,std_flowiat,min_active,mean_active,max_active,std_active,min_idle,mean_idle,max_idle,std_idle,clas
0,9368711.0,16.0,4.0,1564818.0,1549373.0,1.902053e+05,2.032905e+05,3.898224e+05,3.703237e+05,10.353612,...,2.676002e+05,1871488.0,1.983656e+06,2195089.0,1.832197e+05,1234883.0,1420565.0,1523088.0,1.610965e+05,2
1,7340238.0,18.0,4.0,1567554.0,1527893.0,1.656870e+05,1.869148e+05,3.172675e+05,3.043707e+05,11.580006,...,2.214629e+05,1491627.0,3.572433e+06,5653239.0,2.942704e+06,1131498.0,1324636.0,1517774.0,2.731384e+05,2
2,4644225.0,29.0,15.0,1270547.0,1079974.0,1.658652e+05,1.953021e+05,3.294731e+05,3.004926e+05,11.412022,...,2.174754e+05,1758922.0,1.758922e+06,1758922.0,0.000000e+00,1079974.0,1079974.0,1079974.0,0.000000e+00,2
3,4978735.0,19.0,8.0,2492050.0,2457286.0,2.395432e+05,2.765964e+05,6.124353e+05,6.283396e+05,8.034169,...,4.369597e+05,1710925.0,2.382905e+06,3054885.0,9.503232e+05,1346073.0,1894031.5,2441990.0,7.749303e+05,2
4,11838189.0,19.0,10.0,3094089.0,3093543.0,2.437665e+05,2.959547e+05,5.997218e+05,6.256327e+05,7.602514,...,4.361296e+05,1747431.0,2.400446e+06,3240696.0,6.232744e+05,1394455.0,1983227.0,3042717.0,7.259878e+05,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,10006782.0,3999253.0,3999290.0,6007321.0,6007291.0,5.003287e+06,5.003290e+06,1.419918e+06,1.419871e+06,0.599593,...,2.830650e+06,3999290.0,5.003290e+06,6007291.0,1.419871e+06,3999052.0,5003067.5,6007083.0,1.419892e+06,5
17969,14012396.0,6008673.0,6008650.0,8003530.0,8003564.0,7.006102e+06,7.006107e+06,1.410577e+06,1.410617e+06,0.428192,...,3.901489e+06,6008650.0,7.006107e+06,8003564.0,1.410617e+06,6008491.0,7005931.0,8003371.0,1.410593e+06,5
18196,8712389.0,1990162.0,1988563.0,6608031.0,6609078.0,4.299096e+06,4.298820e+06,3.265326e+06,3.267197e+06,0.688674,...,2.763989e+06,1990162.0,4.299096e+06,6608031.0,3.265326e+06,1875414.0,4185148.0,6494882.0,3.266457e+06,5
18261,14253939.0,7002496.0,6995059.0,7006795.0,7042458.0,7.004646e+06,7.018758e+06,3.039852e+03,3.351615e+04,0.420936,...,3.599758e+06,7002496.0,7.004646e+06,7006795.0,3.039852e+03,6790373.0,6794091.5,6797810.0,5.258753e+03,5


In [12]:
class_fraction(none_missing, orig)

,population,fraction
clas,,
BROWSING (1),1678,67.1
CHAT (2),482,54.1
FT (3),193,18.9
MAIL (4),77,30.9
P2P (5),60,6.0
STREAMING (6),121,25.1
VOIP (7),13,0.4
VPN-BROWSING (8),1321,52.8
VPN-CHAT (9),751,62.7


It does not help that is applied to more than a half of the rows

We could try to take `none_missing` as the base and continue with it. But it is very unbalances in terms of classes population. In such a case we will have to remove some categories. At least *VOIP.

We also could do something with the missing values.

Like substitute some features with binary features (present/not present)

In [13]:
def val_types_counts(df):
    return df.melt(id_vars=['count'], value_vars=list(set(df.columns) - {"count"})).groupby(["variable", "value"]).sum()

val_types_orig = h.to_val_types(orig.drop(columns="clas"))
val_types_counts(val_types_orig)

count
variable           value       
duration            0       532
                    1     18226
flowBytesPerSecond  0       532
                    1     18226
flowPktsPerSecond   0       532
                    1     18226
max_active         -1     11355
                    1      7403
max_biat            0      3637
                    1     15121
max_fiat            0      2641
                    1     16117
max_flowiat        -1       532
                    1     18226
max_idle           -1     11355
                    1      7403
mean_active         0     11355
                    1      7403
mean_biat           0      5034
                    1     13724
mean_fiat           0      4640
                    1     14118
mean_flowiat        0       532
                    1     18226
mean_idle           0     11355
                    1      7403
min_active         -1     11355
                    1      7403
min_biat           -1      3637
                    1     15121
min_fiat           -1      2641
                    1     16117
min_flowiat        -1       708
                    0       913
                    1     17137
min_idle           -1     11355
                    1      7403
std_active          0     14935
                    1      3823
std_flowiat         0      2926
                    1     15832
std_idle            0     14935
                    1      3823
total_biat         -1      3637
                    0       520
                    1     14601
total_fiat         -1      2641
                    0       203
                    1     15914

In [14]:
no_dur = orig.loc[orig["duration"] == 0]
h.describe(no_dur)

,count,mean,std,min,25%,50%,75%,max
duration,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
total_fiat,532.0,NaN,-inf,NaN,NaN,NaN,NaN,NaN
total_biat,532.0,NaN,-inf,NaN,NaN,NaN,NaN,NaN
min_fiat,532.0,NaN,-inf,NaN,NaN,NaN,NaN,NaN
min_biat,532.0,NaN,-inf,NaN,NaN,NaN,NaN,NaN
max_fiat,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
max_biat,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
mean_fiat,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
mean_biat,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf
flowPktsPerSecond,532.0,-inf,-inf,-inf,-inf,-inf,-inf,-inf


In [15]:
class_fraction(no_dur, orig)

,population,fraction
clas,,
BROWSING (1),NaN,NaN
CHAT (2),2.0,0.2
FT (3),156.0,15.3
MAIL (4),4.0,1.6
P2P (5),7.0,0.7
STREAMING (6),31.0,6.4
VOIP (7),6.0,0.2
VPN-BROWSING (8),NaN,NaN
VPN-CHAT (9),33.0,2.7


`duration == 0` should probably not be considered a valid input.

If there is no information about the flow, how can we call it a flow?

TODO: Mostly FT and VPN-P2P. Can it be used?

In [16]:
dur = orig.loc[orig["duration"] != 0]
class_fraction(dur, orig)

,population,fraction
clas,,
BROWSING (1),2500,100.0
CHAT (2),888,99.7
FT (3),862,84.6
MAIL (4),245,98.3
P2P (5),993,99.3
STREAMING (6),451,93.5
VOIP (7),2820,99.7
VPN-BROWSING (8),2500,100.0
VPN-CHAT (9),1163,97.2


In [17]:
val_types_raw = h.to_val_types(dur.drop(columns="clas"))
val_types_raw.drop(columns="count").transpose().apply(lambda x: x.unique(), axis=1)

duration                     [1]
total_fiat            [1, -1, 0]
total_biat            [1, -1, 0]
min_fiat                 [1, -1]
min_biat                 [1, -1]
max_fiat                  [1, 0]
max_biat                  [1, 0]
mean_fiat                 [1, 0]
mean_biat                 [1, 0]
flowPktsPerSecond            [1]
flowBytesPerSecond           [1]
min_flowiat           [1, 0, -1]
max_flowiat                  [1]
mean_flowiat                 [1]
std_flowiat               [1, 0]
min_active               [-1, 1]
mean_active               [0, 1]
max_active               [-1, 1]
std_active                [0, 1]
min_idle                 [-1, 1]
mean_idle                 [0, 1]
max_idle                 [-1, 1]
std_idle                  [0, 1]
dtype: object

TODO: how about using only `["duration", "flowPktsPerSecond", "flowBytesPerSecond", "max_flowiat", "mean_flowiat"]`. Those are present whenever duration is not zero. Which is - in the most of the columns.

In [18]:
val_types = val_types_raw.drop(columns=["duration", "flowPktsPerSecond", "flowBytesPerSecond", "max_flowiat", "mean_flowiat"])
val_types.drop(columns="count").transpose().apply(lambda x: x.unique(), axis=1)

total_fiat     [1, -1, 0]
total_biat     [1, -1, 0]
min_fiat          [1, -1]
min_biat          [1, -1]
max_fiat           [1, 0]
max_biat           [1, 0]
mean_fiat          [1, 0]
mean_biat          [1, 0]
min_flowiat    [1, 0, -1]
std_flowiat        [1, 0]
min_active        [-1, 1]
mean_active        [0, 1]
max_active        [-1, 1]
std_active         [0, 1]
min_idle          [-1, 1]
mean_idle          [0, 1]
max_idle          [-1, 1]
std_idle           [0, 1]
dtype: object

TODO: inspect feature that turn both into 0 and -1, does not make much sense to me.
If it is only one of them - then it is probably a default for unknown, two of them are strange

In [19]:
conn = h.get_connections(val_types)
conn_df = h.get_connections_df(conn)

print(conn_df.shape)

(143, 4)


In [20]:
h.get_connection_groups(conn)

[{('max_fiat', 0), ('min_fiat', -1), ('total_fiat', -1)},
 {('max_active', 1),
  ('max_idle', 1),
  ('mean_active', 1),
  ('mean_idle', 1),
  ('min_active', 1),
  ('min_idle', 1)},
 {('std_active', 0), ('std_idle', 0)},
 {('max_fiat', 1), ('min_fiat', 1)},
 {('max_active', -1),
  ('max_idle', -1),
  ('mean_active', 0),
  ('mean_idle', 0),
  ('min_active', -1),
  ('min_idle', -1)},
 {('max_biat', 0), ('min_biat', -1), ('total_biat', -1)},
 {('max_biat', 1), ('min_biat', 1)},
 {('std_active', 1), ('std_idle', 1)}]

Substituting some things to flags

TODO: 

In [21]:
active_flags = dur.copy()
active_flags["has_active"] = active_flags["max_active"] > 0
active_flags["has_std_active"] = active_flags["std_active"] > 0
active_flags = active_flags.drop(columns=["max_active", "max_idle", "mean_active", "mean_idle", "min_active", "min_idle", "std_active", "std_idle"])
active_flags

,duration,total_fiat,total_biat,min_fiat,min_biat,max_fiat,max_biat,mean_fiat,mean_biat,flowPktsPerSecond,flowBytesPerSecond,min_flowiat,max_flowiat,mean_flowiat,std_flowiat,clas,has_active,has_std_active
0,9368711.0,16.0,4.0,1564818.0,1549373.0,190205.285714,203290.456522,389822.391917,370323.719754,10.353612,4802.688438,4.0,1523088.0,97590.739583,267600.198443,2,True,True
1,7340238.0,18.0,4.0,1567554.0,1527893.0,165686.977273,186914.846154,317267.548742,304370.651301,11.580006,4340.186245,4.0,1517774.0,87383.785714,221462.862028,2,True,True
2,4644225.0,29.0,15.0,1270547.0,1079974.0,165865.178571,195302.130435,329473.126261,300492.588227,11.412022,4161.512416,15.0,1079974.0,89312.019231,217475.425246,2,True,False
3,4978735.0,19.0,8.0,2492050.0,2457286.0,239543.250000,276596.388889,612435.304238,628339.573544,8.034169,3918.666087,8.0,2441990.0,127659.871795,436959.716436,2,True,True
4,11838189.0,19.0,10.0,3094089.0,3093543.0,243766.500000,295954.725000,599721.781709,625632.703972,7.602514,2802.540152,10.0,3042717.0,133013.359551,436129.639296,2,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18753,73240.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,27.307482,6116.876024,73240.0,73240.0,73240.000000,0.000000,5,False,False
18754,52083.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,38.400246,8601.655051,52083.0,52083.0,52083.000000,0.000000,5,False,False
18755,67923.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,29.445107,6595.703959,67923.0,67923.0,67923.000000,0.000000,5,False,False
18756,313588.0,-1.0,-1.0,-1.0,-1.0,0.000000,0.000000,0.000000,0.000000,6.377795,1428.626095,313588.0,313588.0,313588.000000,0.000000,5,False,False


In [22]:
val_types = h.to_val_types(active_flags.drop(columns="clas")).drop(
    columns=[
        "duration",
        "flowPktsPerSecond",
        "flowBytesPerSecond",
        "max_flowiat",
        "mean_flowiat",
    ]
)
val_types.drop(columns="count").transpose().apply(lambda x: x.unique(), axis=1)

total_fiat        [1, -1, 0]
total_biat        [1, -1, 0]
min_fiat             [1, -1]
min_biat             [1, -1]
max_fiat              [1, 0]
max_biat              [1, 0]
mean_fiat             [1, 0]
mean_biat             [1, 0]
min_flowiat       [1, 0, -1]
std_flowiat           [1, 0]
has_active            [0, 1]
has_std_active        [0, 1]
dtype: object